In [1]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define variables for threading and data storage
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Function to handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke event triggered.")
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke event triggered.")
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
        q.put(f"{port_identifier} - Left poke with pellet event triggered.")
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
        q.put(f"{port_identifier} - Right poke with pellet event triggered.")
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

# Port detection with logging
def get_device_mappings_by_usb_port():
    logging.info("Starting USB device mapping...")
    device_mappings = []
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
    }
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        try:
            serial_port = os.path.realpath(symlink_path)
            logging.debug(f"Checking symlink path: {symlink_path} -> Serial port: {serial_port}")
            if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
                usb_port_path = get_usb_port_path_from_symlink(symlink)
                if usb_port_path and usb_port_path in usb_port_mapping:
                    port_identifier = usb_port_mapping[usb_port_path]
                    device_mappings.append({
                        'serial_port': serial_port,
                        'port_identifier': port_identifier,
                    })
                    logging.info(f"Device mapped: {serial_port} as {port_identifier}")
                else:
                    logging.warning(f"No mapping found for USB path: {usb_port_path}")
            else:
                logging.warning(f"Unsupported device type for serial port: {serial_port}")
        except Exception as e:
            logging.error(f"Error processing symlink {symlink}: {e}")
    if not device_mappings:
        logging.error("No devices detected. Please check connections.")
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

# Function to read from serial port (FED3 devices) with error handling
def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
        q.put("Ready")
    except serial.SerialException as e:
        q.put(f"Error opening serial port: {e}")
        logging.error(f"Error opening serial port {serial_port}: {e}")
        return
    try:
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list[0] = timestamp
                    process_event(event_type, port_identifier, gpio_pins, q)
                    q.put(data_list)
    finally:
        ser.close()

# Splash screen class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)

        screen_width = 800
        screen_height = 480
        width, height = 600, 200
        x, y = (screen_width - width) // 2, (screen_height - height) // 2
        self.root.geometry(f"{width}x{height}+{x}+{y}")
        self.root.configure(bg="black")

        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Helvetica", 24, "bold"), bg="black", fg="orange")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

# Main GUI Application Class
class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.01")
        self.root.geometry("800x480")

        # Configure main frame
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))

        # Port widgets, queues, and controls
        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {port: [] for port in gpio_pins_per_device}  # Initialize data for each port
        self.threads = []

        # Add port display and controls
        self.setup_ports()
        self.create_controls()
        self.root.after(100, self.update_gui)

        # Attempt to detect devices immediately
        self.check_connected_devices()

    def setup_ports(self):
        port_names = ['Port 1', 'Port 2', 'Port 3', 'Port 4']
        for idx, port_name in enumerate(port_names):
            frame = ttk.LabelFrame(self.mainframe, text=port_name, padding="3")
            frame.grid(column=idx % 2, row=idx // 2, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 10), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_name] = {'frame': frame, 'status_label': status_label, 'text_widget': text_widget}
            self.port_queues[port_name] = queue.Queue()

    def create_controls(self):
        tk.Label(self.mainframe, text="Experimenter:", font=("Helvetica", 8)).grid(column=0, row=3, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name, width=12)
        self.experimenter_entry.grid(column=1, row=3, sticky=tk.W)

        tk.Label(self.mainframe, text="Experiment Name:", font=("Helvetica", 8)).grid(column=2, row=3, sticky=tk.W)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name, width=12)
        self.experiment_entry.grid(column=3, row=3, sticky=tk.W)

        self.start_button = tk.Button(self.mainframe, text="START", font=("Helvetica", 10, "bold"), bg="green", command=self.start_experiment)
        self.start_button.grid(column=0, row=4, padx=5, pady=5)

        self.stop_button = tk.Button(self.mainframe, text="STOP", font=("Helvetica", 10, "bold"), bg="red", command=self.stop_experiment)
        self.stop_button.grid(column=1, row=4, padx=5, pady=5)

        self.browse_button = tk.Button(self.mainframe, text="Browse Data Folder", font=("Helvetica", 10), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=2, row=4, padx=5, pady=5, columnspan=2, sticky=tk.E)

        # Add recording indicator canvas
        self.canvas = tk.Canvas(self.mainframe, width=100, height=100)
        self.canvas.grid(column=3, row=4, padx=5, pady=5)
        self.recording_circle = None

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        self.device_mappings = get_device_mappings_by_usb_port()
        for mapping in self.device_mappings:
            port_identifier = mapping['port_identifier']
            self.port_widgets[port_identifier]['status_label'].config(text="Ready", font=("Helvetica", 10), foreground="green")
            logging.info(f"{port_identifier} set to Ready.")

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def start_experiment(self):
        if not self.device_mappings:
            logging.warning("No devices connected. Cannot start experiment.")
            return
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)
        
        # Display recording indicator
        self.display_recording_indicator()

    def display_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(10, 10, 90, 90, fill="red")
            self.canvas.create_text(50, 120, text="RECORDING STARTED", font=("Helvetica", 10, "bold"))

    def hide_recording_indicator(self):
        if self.recording_circle:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None

    def stop_experiment(self):
        stop_event.set()
        for t in self.threads:
            t.join()  # Ensure threads stop cleanly
        GPIO.cleanup()
        self.save_all_data()
        self.hide_recording_indicator()
        self.root.quit()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()
        
        if not self.save_path or not experimenter_name or not experiment_name:
            logging.warning("Incomplete information. Data will not be saved.")
            return
        
        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, experiment_name)
        os.makedirs(experiment_folder, exist_ok=True)

        for port_identifier, data_rows in self.data_to_save.items():
            filename = f"{experiment_folder}/{port_identifier}_{current_time}.csv"
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

# Main execution
if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


2024-11-04 14:49:24,461 [INFO] Starting USB device mapping...
2024-11-04 14:49:24,466 [DEBUG] Checking symlink path: /dev/serial/by-path/platform-fd500000.pcie-pci-0000:01:00.0-usb-0:1.3:1.0 -> Serial port: /dev/ttyACM0
2024-11-04 14:49:24,471 [INFO] Device mapped: /dev/ttyACM0 as Port 3
2024-11-04 14:49:24,477 [INFO] Port 3 set to Ready.
2024-11-04 14:50:00,566 [INFO] Opened serial port /dev/ttyACM0 for Port 3
2024-11-04 14:50:03,976 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:06,594 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:10,290 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:11,758 [DEBUG] Sending TTL signal to pin 0
2024-11-04 14:50:28,600 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:29,776 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:32,611 [DEBUG] Sending TTL signal to pin 0
2024-11-04 14:50:33,944 [DEBUG] Sending TTL signal to pin 0
2024-11-04 14:50:37,572 [DEBUG] Sending TTL signal to pin 5
2024-11-04 14:50:38,687 [DEBUG] Sending TTL s